In [1]:
import pandas as pd
df=pd.read_csv('obesity_IP6K3.csv')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.linear_model import LinearRegression
from testset_prediction import testset_prediction as tsp
from sequential_selection import stepwise_selection as sq
from loo import loo
from rm2 import rm2
reg=LinearRegression()

num_des=5   #Specify the number of descriptors to be selected in the model

def process(tr,trX,Xts,y,yts,seed):
    lt=[]
    ls=['r2','neg_mean_absolute_error','neg_mean_poisson_deviance','neg_mean_gamma_deviance']
    #ls=['r2']
    lf=[0,5]
    #lf=[0]
    l1,l2,l3,l4,l5,l6,l7=[],[],[],[],[],[],[]
    xl=[]
    for i in ls:
        for j in lf:
            sqs=sq(trX,y,num_des,True,True,i,j)
            try:
               a1,b1=sqs.fit_()
               reg.fit(tr[a1],y)
               r2tr=reg.score(tr[a1],y)
               cv=loo(tr[a1],y,tr)
               xl.append(a1)
               print(str(len(xl))+' Model generated')
               c,m,l=cv.cal()
               r2pr,r2pr2,RMSEP=testpred(Xts[a1],yts,reg,m)
               rm2tr,drm2tr=rm2(y,l).fit()
               ytspr=pd.DataFrame(reg.predict(Xts[a1]))
               rm2ts,drm2ts=rm2(yts,ytspr).fit()
            except ValueError:
                c=0
                r2pr=0
                rm2tr=0
                rm2ts=0         
            #print(c,m,l)
            #print(a1)
            l1.append(i) 
            l2.append(j)  
            l3.append(c)
            l4.append(r2pr)
            l6.append(rm2tr)
            l7.append(rm2ts)
            l5.append(seed)
    Dict=dict([('random_seed', l5),('score', l1),('fold', l2),('Q2LOO', l3), ('R2Pred', l4), ('rm2tr', l6), ('rm2ts', l7)])
    #print(Dict)
    table=pd.DataFrame(Dict)
    return table
    #tbname='Results_table.csv'
    #table.to_csv('Results_table.csv',index=False)  

In [5]:
from sklearn.feature_selection import VarianceThreshold
def correlation(X,cthreshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = X.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (abs(corr_matrix.iloc[i, j]) > float(cthreshold)) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in X.columns:
                   del X[colname] # deleting the column from the dataset
    return X   

def variance(X,threshold):
    sel = VarianceThreshold(threshold=(threshold* (1 - threshold)))
    sel_var=sel.fit_transform(X)
    X=X[X.columns[sel.get_support(indices=True)]]    
    return X


def pretreat(X,cthreshold,vthreshold):
    X=correlation(X,cthreshold)
    X=variance(X,vthreshold)
    return X

In [6]:
def testpred(Xts,yts,model,trav):
    ytspr=pd.DataFrame(model.predict(Xts))
    ytspr.columns=['Pred']
    tsdf=pd.concat([yts,pd.DataFrame(ytspr)],axis=1)
    tsdf.columns=['Active','Predict']
    tsdf['Aver']=trav
    tsdf['Aver2']=tsdf['Predict'].mean()
    tsdf['diff']=tsdf['Active']-tsdf['Predict']
    tsdf['diff2']=tsdf['Active']-tsdf['Aver']
    tsdf['diff3']=tsdf['Active']-tsdf['Aver2']
    r2pr=1-((tsdf['diff']**2).sum()/(tsdf['diff2']**2).sum())
    r2pr2=1-((tsdf['diff']**2).sum()/(tsdf['diff3']**2).sum())
    RMSEP=((tsdf['diff']**2).sum()/tsdf.shape[0])**0.5
    return r2pr,r2pr2,RMSEP

In [7]:
import pandas as pd
X=df.iloc[:,2:]
Xpt=pretreat(X,0.99,0.0001)

In [ ]:
Xpt.to_csv('Pretreated_0.99_0.0001_RD.csv')

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd
#perc=secondEntryTabOne.get()
#df=pd.read_csv('merged2_pparg_model2.csv')
#X=df.iloc[:,1:]
#pt=pd.read_csv('pt_train_0.99_0.0001.csv')
Xpt2=Xpt.columns.tolist()[1:]
perc=0.2
nls=[]
ml=[3,20,42]
#ml=[20]
for rs in ml:
    print('Random seed: '+str(rs)+': ')
    #perc=int(secondEntryTabOne.get())
    #pc=int(100/perc)
    #sp=int(secondEntryTabOne_x.get())
    #filem=df.sort_values(df.iloc[:,1:2].columns[0],ascending=False)
    #ts=filem.iloc[(sp-1)::pc, :]
    #tr=filem.drop(ts.index.values)
    #tr.to_csv('tr.csv', index=False)
    a,b= train_test_split(df,test_size=perc, random_state=rs)
    tr=pd.DataFrame(a)
    ts=pd.DataFrame(b)
    tr=tr.reset_index().drop('index', axis=1)
    ts=ts.reset_index().drop('index', axis=1)
    trX=tr[Xpt2]
    Xts=ts[Xpt2]
    #print(trX.shape)
    y=tr.iloc[:,1:2]
    tr1=pd.concat([tr.iloc[:,0:2],trX],axis=1)
    ts1=pd.concat([ts.iloc[:,0:2],Xts],axis=1)
    #tr1.to_csv('tr.csv', index=False)
    #ts1.to_csv('ts.csv', index=False)
    yts=ts.iloc[:,1:2] 
    table=process(tr,trX,Xts,y,yts,rs)
    nls.append(table)
    fd=pd.concat(nls, axis=0)

Random seed: 3: 
1 Model generated
2 Model generated
3 Model generated
4 Model generated
5 Model generated
6 Model generated
7 Model generated
8 Model generated
Random seed: 20: 
1 Model generated
2 Model generated
3 Model generated
4 Model generated
5 Model generated
6 Model generated
7 Model generated
8 Model generated
Random seed: 42: 
1 Model generated
2 Model generated
3 Model generated
4 Model generated
5 Model generated
6 Model generated
7 Model generated
8 Model generated


In [9]:
fd['Average']=fd[["Q2LOO","R2Pred"]].mean(axis=1)
fd=fd.sort_values('Average', ascending=False)

In [10]:
fd

,random_seed,score,fold,Q2LOO,R2Pred,rm2tr,rm2ts,Average
7,20,neg_mean_gamma_deviance,5,0.904968,0.502389,0.854523,0.465108,0.703678
2,20,neg_mean_absolute_error,0,0.813628,0.521918,0.748770,0.492456,0.667773
3,20,neg_mean_absolute_error,5,0.643678,0.663003,0.537452,0.569851,0.653341
1,20,r2,5,0.827483,0.414628,0.759393,0.387882,0.621056
5,20,neg_mean_poisson_deviance,5,0.880607,0.242158,0.832180,0.357049,0.561382
1,42,r2,5,0.917105,0.140405,0.882786,0.117846,0.528755
4,20,neg_mean_poisson_deviance,0,0.942297,0.058793,0.916846,0.287788,0.500545
0,20,r2,0,0.942297,0.058793,0.916846,0.287788,0.500545
2,3,neg_mean_absolute_error,0,0.864568,0.021726,0.815121,0.136973,0.443147
6,20,neg_mean_gamma_deviance,0,0.805180,-0.101825,0.712339,0.241416,0.351678


In [11]:
fd.to_csv('random division results.csv', index=False) #change output file name